# Install packages


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.5.0 bitsandbytes==0.40.2 trl==0.4.7 langchain
!pip install -q --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Make sure a GPU is available
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

GPU is available


# Imports

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pandas as pd
from string import Template
from pathlib import Path

import os

import warnings
warnings.simplefilter("ignore")

from tqdm.notebook import tqdm

# for training
from peft import LoraConfig, get_peft_model, LoraModel, PeftModel
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# for traing set
from datasets import load_dataset
from langchain.prompts import PromptTemplate
import matplotlib.pyplot as plt
import bitsandbytes as bnb
import numpy as np

from IPython.display import Markdown, display

In [ ]:
# Log in to huggingface to load/push fine-tuned models in hub
from huggingface_hub import notebook_login

notebook_login()

# Load model and tokenizer


In [ ]:
# Change model_name to the model of your choice
# This can be either name of the model on huggingface (requires internet) or path to the model
# model_name = "/kaggle/input/llama2-7b-hf/Llama2-7b-hf"

# Tried the first three, failed at the training step
# model_name = "NousResearch/llama-2-7b-chat-hf"
# model_name = "guardrail/llama-2-7b-guanaco-instruct-sharded"
# model_name = "abhishek/llama-2-7b-hf-small-shards"

# model_name = "lmsys/vicuna-7b-v1.5"
model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtyp=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
# this should be set as False for finetuning
model.config.use_cache = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# For llama2 models
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# For Vicuna model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

# Prepare for Inference

### Prepare test set

In [ ]:
# Same prompt as before
template = """Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

Question: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n
E) {e}\n

### Answer: {answer}"""

prompt = PromptTemplate(template=template, input_variables=['prompt', 'a', 'b', 'c', 'd', 'e', 'answer'])

In [ ]:
# We don't give answers for test
def format_text_test(example):
    text = prompt.format(prompt=example['prompt'],
                         a=example['A'],
                         b=example['B'],
                         c=example['C'],
                         d=example['D'],
                         e=example['E'],
                         answer='')
    return {"text": text}


test_dataset = load_dataset("csv", data_files="/content/thesis_questions_test.csv")
test_dataset = test_dataset.map(format_text_test)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

In [ ]:
# Create a test set dataframe
csv_file_path = '/content/thesis_questions_test.csv'

# Replace these with the actual column names you want to keep
columns_to_keep = ['index', 'answer']

# Read the CSV file using pandas and select only the specified columns
test_df = pd.read_csv(csv_file_path, usecols=columns_to_keep)

# Now df will contain only the columns listed in columns_to_keep
print(test_df.head())

   index answer
0    199      B
1    789      E
2    174      B
3    467      A
4     66      D


### Predict the next token and do perplexity ranking

In [ ]:
# Define the metric of perplexity
from torch import nn

class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss() # CE loss is the average negative log-likelihood of the next token appearing given the input sequence
        self.reduce = reduce # Control whether the computed perplexity should be reduced (averaged) across all examples in a batch

    def forward(self, logits, labels):
        shift_logits = logits[..., :-1, :].contiguous() # Remove the last token's logits because there is no next token to predict after the last one
        shift_labels = labels[..., 1:].contiguous() # Shift the labels by one position to the left, aligning each token with its subsequent token as the label for prediction

        perplexity = []
        for i in range(labels.shape[0]):
            perplexity.append(self.loss_fn(shift_logits[i], shift_labels[i]))
        perplexity = torch.stack(perplexity, dim=0)
        if self.reduce:
            perplexity = torch.mean(perplexity)
        return perplexity

perp = Perplexity()

In [ ]:
def evaluate_model_on_dataset(model, tokenizer, dataset, device="cuda"):
    model.eval()
    preds = []
    perplexities_ordered = []  # List to store ordered perplexity values for each option

    for idx in tqdm(range(len(dataset["train"])), total=len(dataset["train"])):
        with torch.no_grad():
            # Prepare the sample inputs
            cols = ["A", "B", "C", "D", "E"]
            perps = []
            samples = [dataset['train'][idx]['text'] + col for col in cols]

            # Tokenize the samples
            inputs = tokenizer(samples, return_tensors="pt", add_special_tokens=False, padding=True, truncation=True).to(device)

            # Get model output
            output = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
            logits = output.logits
            labels = inputs["input_ids"] # Set up the ground truth for the next-token prediction
            labels.masked_fill_(~inputs["attention_mask"].bool(), -100) # Mask out the tokens that the model should not consider for loss calculation, only keep the option label

            # Calculate perplexity for each option
            perp = Perplexity().to(device)
            for j in range(len(cols)):
                p = perp(logits[j].unsqueeze(0), labels[j].unsqueeze(0))
                perps.append(p.item())

            # Memory management: delete variables no longer needed
            del inputs, labels, output, p

            # Sort the indices based on perplexity and get the corresponding options and perplexities
            sorted_indices = np.argsort(perps)
            predictions = [cols[i] for i in sorted_indices]
            sorted_perps = [perps[i] for i in sorted_indices]

            preds.append(predictions)
            perplexities_ordered.append(sorted_perps)

    return preds, perplexities_ordered


# Inference

### Inference with pre-trained model

In [ ]:
old_preds, old_perplexities_ordered = evaluate_model_on_dataset(model, tokenizer, test_dataset)

  0%|          | 0/195 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(old_preds)
print(old_perplexities_ordered)

[['B', 'E', 'C', 'A', 'D'], ['E', 'A', 'D', 'B', 'C'], ['C', 'B', 'E', 'D', 'A'], ['B', 'A', 'E', 'C', 'D'], ['B', 'A', 'E', 'C', 'D'], ['A', 'B', 'C', 'E', 'D'], ['D', 'E', 'C', 'B', 'A'], ['D', 'C', 'B', 'A', 'E'], ['B', 'C', 'D', 'E', 'A'], ['B', 'C', 'E', 'D', 'A'], ['B', 'A', 'D', 'E', 'C'], ['B', 'E', 'D', 'C', 'A'], ['A', 'D', 'B', 'E', 'C'], ['B', 'C', 'D', 'A', 'E'], ['D', 'C', 'E', 'B', 'A'], ['C', 'B', 'E', 'D', 'A'], ['A', 'B', 'E', 'D', 'C'], ['B', 'A', 'C', 'E', 'D'], ['E', 'D', 'B', 'C', 'A'], ['E', 'D', 'A', 'C', 'B'], ['B', 'A', 'D', 'E', 'C'], ['C', 'D', 'B', 'A', 'E'], ['B', 'A', 'E', 'C', 'D'], ['B', 'D', 'C', 'E', 'A'], ['B', 'A', 'C', 'E', 'D'], ['E', 'D', 'B', 'C', 'A'], ['E', 'C', 'B', 'D', 'A'], ['E', 'C', 'D', 'B', 'A'], ['B', 'C', 'A', 'E', 'D'], ['E', 'B', 'D', 'C', 'A'], ['E', 'B', 'D', 'C', 'A'], ['A', 'B', 'D', 'E', 'C'], ['B', 'C', 'E', 'D', 'A'], ['B', 'E', 'D', 'C', 'A'], ['B', 'D', 'C', 'A', 'E'], ['A', 'B', 'E', 'D', 'C'], ['B', 'C', 'E', 'D', 'A'], 

In [ ]:
def format_prediction(prediction_list, k=3):
    """Formats the top-k predictions into a single string."""
    return ' '.join(prediction_list[:k])

In [ ]:
# Change column names
# top1_col_name = 'vicuna_7B_pt_top1'
# top3_col_name = 'vicuna_7B_pt_top3'
top1_col_name = 'llama2_7B_chat_pt_top1'
top3_col_name = 'llama2_7B_chat_pt_top3'

preds = old_preds

# Create new columns for k=1 and k=3 formatted predictions
test_df[top1_col_name] = [format_prediction(pred, k=1) for pred in preds]
test_df[top3_col_name] = [format_prediction(pred, k=3) for pred in preds]

print(test_df.head())

   index answer llama2_7B_chat_pt_top1 llama2_7B_chat_pt_top3
0    199      B                      B                  B E C
1    789      E                      E                  E A D
2    174      B                      C                  C B E
3    467      A                      B                  B A E
4     66      D                      B                  B A E


In [ ]:
# Save for safety
test_df.to_csv('/content/test_results.csv', index = False)

# Evaluation

In [ ]:
test_df = pd.read_csv("/content/test_results.csv")
test_df.head()

,index,answer,llama2_7B_chat_pt_top1,llama2_7B_chat_pt_top3
0,199,B,B,B E C
1,789,E,E,E A D
2,174,B,C,C B E
3,467,A,B,B A E
4,66,D,B,B A E


In [ ]:
# test_df = pd.read_csv("/content/test_results.csv")
# test_df.head()

,index,answer,llama2_7B_chat_pt_top1,llama2_7B_chat_pt_top3
0,199,B,B,B E C
1,789,E,E,E A D
2,174,B,C,C B E
3,467,A,B,B A E
4,66,D,B,B A E


In [ ]:
# Compute the Mean Average Precision (MAP) for the top 3 choices

def precision_at_k(relevance, k):
    assert k <= len(relevance), "k is larger than the length of the list of predictions"
    return sum(relevance[:k]) / k

def MAP_at_3(df, true_col, prediction_col):
    map_score = 0.0
    for _, row in df.iterrows():
        predictions = row[prediction_col].split()  # Split the predictions into a list
        true_answer = row[true_col]
        user_results = [1 if pred == true_answer else 0 for pred in predictions[:3]]

        for k in range(min(len(predictions), 3)):
            # Only consider the first correct label, ignore the rest for MAP calculation
            if user_results[k] == 1:
                map_score += precision_at_k(user_results, k+1)
                break  # Stop after the first correct prediction
    return map_score / len(df)


In [ ]:
def calculate_accuracy(df, true_col, pred_col):
    # Comparing the true answers with the top-1 predictions based on perplexity
    correct_predictions = (df[true_col] == df[pred_col])
    # Calculating the accuracy
    accuracy = correct_predictions.sum() / len(df)
    return accuracy

In [ ]:
# Compute the accuracy
old_accuracy = calculate_accuracy(test_df, 'answer', top1_col_name)
print(f"Old Accuracy: {old_accuracy}")

# Compute the MAP@3 scores
old_map_at_3_score = MAP_at_3(test_df, 'answer', top3_col_name)
print(f"Old MAP at 3: {old_map_at_3_score}")

Old Accuracy: 0.36923076923076925
Old MAP at 3: 0.5230769230769229


In [ ]:
# Compute the accuracy
old_accuracy = calculate_accuracy(test_df, 'answer', top1_col_name)
print(f"Old Accuracy: {old_accuracy}")

# Compute the MAP@3 scores
old_map_at_3_score = MAP_at_3(test_df, 'answer', top3_col_name)
print(f"Old MAP at 3: {old_map_at_3_score}")

Old Accuracy: 0.36923076923076925
Old MAP at 3: 0.5230769230769229
